In [1]:
import cv2
import numpy as np 
import os
import yaml
from yaml.loader import SafeLoader

In [4]:
# load yaml file
with open('data.yaml', mode='r') as f:
    data_yaml=yaml.load(f,Loader=SafeLoader)

labels=data_yaml['names']

print(labels)

['pothole']


In [6]:
# yolo model

yolo =cv2.dnn.readNetFromONNX('./Models3/weights/best.onnx')

yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU) 

In [8]:
img=cv2.imread('./testimage.jpg')
image=img.copy()


# cv2.imshow("image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
row, col, d= image.shape

In [10]:
max_rc=max(row, col)
input_img=np.zeros((max_rc, max_rc,3), dtype=np.uint8)


In [33]:
input_img[0:row, 0:col]=image


input_width_yolo=640
input_heigth_yolo=640

blob=cv2.dnn.blobFromImage(input_img,1/255,(input_width_yolo, input_heigth_yolo),swapRB=True, crop=False)


yolo.setInput(blob)
# preds=yolo.forward()


In [34]:
prediction=yolo.forward()

In [36]:
prediction.shape

(1, 25200, 6)

In [48]:
detection=prediction[0]
detection.shape
boxes=[]
confidences=[]
calsses=[]

In [39]:
img_w, img_h= input_img.shape[:2]

In [44]:
xfactor=img_w/input_width_yolo
yfactor=img_h/input_heigth_yolo

In [49]:
for i in range(len(detection)):
    row=detection[i]
    confidence=row[4]
    if confidence>0.4:
        class_score=row[5:].max()
        class_id=row[5:].argmax()

        if class_score >0.25:
            cx,cy,w,h=row[0:4]
            left=int((cx-0.5*w)*xfactor)
            top=int((cy-0.5*h)*yfactor)
            width=int(w*xfactor)
            height=int(h*yfactor)

            box=np.array([left, top, width, height])

            confidences.append(confidence)
            boxes.append(box)
            calsses.append(class_id)

In [56]:
confidences_np=np.array(confidences).tolist()
confidences_np

[0.5513631105422974,
 0.4722784459590912,
 0.6240922212600708,
 0.4726282060146332,
 0.43560275435447693,
 0.48131227493286133,
 0.5905845165252686,
 0.7804234623908997,
 0.6793760061264038,
 0.8625961542129517,
 0.49074313044548035,
 0.49368199706077576,
 0.5295119285583496,
 0.5230363607406616,
 0.45692408084869385,
 0.6954572200775146,
 0.7821767926216125,
 0.7897525429725647,
 0.6231276392936707,
 0.4335682690143585,
 0.7609425783157349,
 0.4123842120170593,
 0.6292724013328552,
 0.8042777180671692,
 0.4630352258682251,
 0.7442067265510559,
 0.798460066318512,
 0.5368713736534119,
 0.7439602017402649,
 0.7305510640144348,
 0.7188665270805359,
 0.8079325556755066,
 0.8466066718101501,
 0.8085824847221375,
 0.9065179228782654,
 0.7637686729431152,
 0.8073176741600037,
 0.6377394795417786,
 0.7765325903892517,
 0.4928012192249298,
 0.7876493334770203,
 0.5065522193908691,
 0.8557016849517822,
 0.8126640319824219,
 0.793718695640564,
 0.6006677746772766,
 0.5159251689910889,
 0.6206234

In [53]:
boxes_np=np.array(boxes).tolist()
boxes_np

[[206, 68, 37, 12],
 [208, 68, 38, 12],
 [361, 118, 39, 13],
 [360, 117, 39, 13],
 [283, 152, 42, 14],
 [405, 168, 36, 12],
 [405, 167, 35, 13],
 [401, 188, 40, 19],
 [401, 188, 39, 18],
 [402, 187, 39, 18],
 [61, 250, 41, 21],
 [62, 250, 41, 22],
 [208, 67, 36, 12],
 [208, 68, 38, 12],
 [164, 106, 56, 14],
 [165, 111, 52, 18],
 [164, 116, 53, 14],
 [360, 118, 40, 12],
 [360, 118, 39, 12],
 [269, 136, 49, 14],
 [269, 137, 48, 14],
 [270, 138, 48, 14],
 [277, 153, 49, 14],
 [280, 152, 47, 14],
 [406, 167, 36, 13],
 [271, 176, 55, 18],
 [272, 175, 55, 19],
 [405, 167, 35, 13],
 [35, 182, 60, 21],
 [36, 183, 60, 20],
 [270, 176, 55, 19],
 [271, 176, 56, 19],
 [403, 188, 40, 17],
 [402, 188, 38, 17],
 [402, 187, 40, 17],
 [248, 199, 63, 23],
 [250, 200, 60, 22],
 [405, 228, 66, 23],
 [408, 228, 65, 23],
 [408, 227, 64, 23],
 [61, 251, 41, 21],
 [60, 250, 45, 21],
 [60, 251, 43, 21],
 [59, 251, 46, 22],
 [336, 309, 64, 47],
 [337, 308, 62, 47],
 [438, 318, 69, 41],
 [206, 67, 38, 11],
 [208

In [57]:
index= cv2.dnn.NMSBoxes(boxes_np,confidences_np, 0.25, 0.45).flatten()

In [58]:
index

array([100, 196,  34, 111, 124, 151, 149, 146,  82, 136, 131,  54,  64,
        31,  16,  60,  69,  47,  58,  49,  56, 132])

In [59]:
# boxing the detection

In [60]:
for ind in index:
    x,y,w,h=boxes_np[ind]
    bb_conf=int(confidences_np[ind]*100)
    class_id=calsses[ind]
    class_name=labels[class_id]
    text=f'{class_name}:{bb_conf}%'

    cv2.rectangle(image, (x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image, (x,y-30),(x+w,y),(255,255,255),-1)

    cv2.putText(image, text,(x,y-10), cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

In [61]:

cv2.imshow("original", img)
cv2.imshow("yolo predection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [62]:
# process to functions